In [109]:
import pandas as pd

In [110]:
prets = pd.read_csv("prets.csv")
prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014


In [111]:
# calacul du taux d'endettement

prets['taux_endettement'] = round(prets['remboursement'] * 100 / prets['revenu'], 2)

In [112]:
# renommer taux en taux_interet

prets.rename(columns={'taux':'taux_interet'}, inplace=True)

In [113]:
# calculer le cout total du pret

prets['cout_total'] = prets['remboursement'] * prets['duree']

In [114]:
# calculer les bénéfices mensuels réalisés

prets['benefices'] = round((prets['cout_total'] * prets['taux_interet']/100)/(24), 2)

In [115]:
prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57


In [116]:
# Le taux d'endettement autorisé est de 35 %. Pourriez-vous me communiquer le nombre de personnes ayant
# dépassé ce seuil ? Vous pouvez stocker la liste dans une variable appelée `clients_risque`.


nombre_client_risque = prets.loc[prets['taux_endettement'] > 35, :].shape[0]
print('Il y a', nombre_client_risque, 'clients qui ont dépassé le seuil autorisé')

# Pour expliquer rapidement cette ligne de code :
# Dans un premier temps, nous utilisons la fonction `.loc` pour sélectionner l'ensemble des lignes 
# dont le taux d'endettement est supérieur à 35 %.
# Ensuite, nous utilisons la méthode `.shape` pour calculer le nombre de clients concernés.

Il y a 58 clients qui ont dépassé le seuil autorisé


In [117]:
# Même question, mais cette fois ci uniquement sur l’agence Parisienne

nombre_client_risque = prets.loc[(prets['taux_endettement'] > 35) & (prets['ville'] == 'PARIS'),:].shape[0]
print('il y a', nombre_client_risque, "clients qui ont depassé le seuil autorisé a l'agenge parisienne")

il y a 16 clients qui ont depassé le seuil autorisé a l'agenge parisienne


In [118]:
# Pour faciliter le traitement d’éventuelles futures demandes de prêts, pourriez-vous ajouter une variable nommée 
# `risque` qui aurait pour valeur "Oui" si le client est risqué (taux d’endettement > 35 %), et "Non" sinon. Pour ce faire, 
# je vous conseille de créer la variable `risque` en l'initialisant à "Non" (ou "Oui", selon votre préférence), 
# puis de ne modifier que les lignes concernées par la valeur inverse.

prets['risque'] = 'non'
prets.loc[prets['taux_endettement'] > 35, 'risque'] = 'oui'

In [119]:
# Combien de prêts automobiles ont été accordés ? Quel est le coût total moyen de ces derniers ?

prets_auto = prets.loc[prets['type'] == 'automobile', :]
print('nous avons accordé', prets_auto.shape[0], 'prets automobile dont le prix moyen est de', prets_auto['cout_total'].mean(), '€')


# Pour expliquer rapidement ces lignes de code :
# Nous opérons une sélection sur les prêts de type automobile. Le résultat de cette sélection est stocké 
# dans un DataFrame nommé `prets_auto`.
# Nous utilisons ensuite la méthode `.shape`, comme précédemment, pour déterminer le nombre de prêts concernés.
# Nous calculons ensuite la moyenne de la variable `cout_total` dans ce DataFrame nouvellement créé.

nous avons accordé 16 prets automobile dont le prix moyen est de 13665.0 €


In [120]:
# Quel est le bénéfice mensuel total réalisé par l’agence Toulousaine ?

prets.loc[prets['ville'] == 'TOULOUSE','benefices'].sum()

2138.0699999999997

In [121]:
prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices,risque
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99,non
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69,non
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97,non
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71,non
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57,non


In [122]:
# Pourriez vous créez un dataframe profil client pour pallier ce problème ?
# Par profil client, nous entendons un dataframe où il y n'y a qu'une seule ligne par client, avec le résumé de 
# ses informations (somme remboursement, du taux d'endettement, du cout_total et des bénéfices réalisés)

profil_clients = prets.groupby('identifiant')[['remboursement','taux_endettement','cout_total','benefices']].sum()
profil_clients.reset_index(inplace=True)
profil_clients.head()

,identifiant,remboursement,taux_endettement,cout_total,benefices
0,0,1130.05,30.80,271212.0,131.99
1,1,1487.85,28.02,389715.0,206.66
2,2,552.54,29.50,132609.6,53.71
3,3,586.03,34.80,105485.4,44.57
4,4,423.61,28.70,101666.4,51.21


In [123]:
# Recalculez le nombre exact de personnes en situation bancaire risquée à partir du taux d'endettemment :

nombre_client_risque = profil_clients.loc[profil_clients['taux_endettement'] > 35, :].shape[0]
print('Il y a',nombre_client_risque , 'clients qui ont dépassé le seuil autorisé')

Il y a 62 clients qui ont dépassé le seuil autorisé


In [124]:
# Calculez le bénéfice dégagé par chacune de nos agences, par types de prêts. Vous présenterez vos résultats sous
# la forme d'un tableau simple (via un group_by) :

prets.groupby(['ville', 'type']) ['benefices'].sum()

ville      type      
BORDEAUX   immobilier    1681.32
LYON       automobile      47.39
           immobilier    2967.41
MARSEILLE  automobile      61.36
           immobilier    4334.55
NICE       automobile      31.15
           immobilier    2651.60
PARIS      automobile      99.50
           immobilier    9916.85
TOULOUSE   automobile      61.22
           immobilier    2076.85
Name: benefices, dtype: float64

In [126]:
# Pour aller plus loin, on souhaite avoir un tableau à double entrée (via un pivot_table) présentant cette fois ci 
# les bénéfices moyen réalisés par chaque agence, pour chaque type de prêt :

prets = prets.pivot_table(index='ville', columns='type', values='benefices', aggfunc='mean')
prets = prets.reset_index()
prets

# Quelle ville semble la plus intéressante où développer les prêts immobiliers ?
# La ville de Nice (ou Paris dans une moindre mesure). C'est là où la marge réalisée est la plus élevée sur les prêts 
# immobiliers.

type,ville,automobile,immobilier
0,BORDEAUX,NaN,105.082500
1,LYON,15.796667,118.696400
2,MARSEILLE,20.453333,60.202083
3,NICE,15.575000,139.557895
4,PARIS,19.900000,137.734028
5,TOULOUSE,20.406667,86.535417


In [ ]:
# variable data frame = dataframe.drop(columns='id')    exemple comment suprimer une colonne